# Beer Recommender System using Unstructured Data Analyitcs

In [ ]:
# importing required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import contextlib
import warnings

import copy 
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

from datetime import date, timedelta
import datetime

In [ ]:

#Selenium is an open-source tool that automates web browsers.
!pip install selenium
!apt-get -q update   #Used to handle installation and removal of softwares and libraries
!apt install -yq chromium-chromedriver #ChromeDriver is a separate executable that Selenium WebDriver uses to control Chrome.
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
#WebDriver is a browser automation framework that works with open source APIs. 
#The framework operates by accepting commands, sending those commands to a browser, and interacting with applications.
chrome_options = webdriver.ChromeOptions()
#headless means running chrome with chrome.exe
chrome_options.add_argument('--headless')

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

### **Task A**
Extract about 5-6k reviews. 

In [ ]:
from selenium import webdriver
import re
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.maximize_window()
link_list=[]

urls=['https://www.beeradvocate.com/beer/top-rated/','https://www.beeradvocate.com/beer/popular/','https://www.beeradvocate.com/beer/top-new/']

for url in urls:
  driver.get(url)
  # identify elements with tagname <a>
  lnks=driver.find_elements("tag name", "a")
  # traverse list
  #storing all the links of profiles of 250 beers here
  for lnk in lnks:
      #if 'profile' in str(lnk.get_attribute('href')):
      link_str=str(lnk.get_attribute('href'))
      if re.match("https://www.beeradvocate.com/beer/profile/.*/.+",link_str):
        if link_str not in link_list:
          link_list.append(link_str)




In [ ]:
from zmq.constants import NULL
import requests
from bs4 import BeautifulSoup

#create a df to store data
reviews_df=pd.DataFrame(columns=['product_name','product_review','user_rating'])

#for each beer url lin linklist get reviews
for url in link_list:
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  beer_company_name=soup.find('h1').text #name for beer
  user_reviews_all=soup.find_all(attrs={'id': 'rating_fullview_content_2'})
  for review in user_reviews_all:
    rating_with_comment = re.search('<span class="BAscore_norm">(.*?)</span>', str(review))
    review_text = re.search('<div style="margin:10px 0px; padding:10px; border-left:2px solid #A9A9A9; font-size:11pt; line-height:1.4;">(.*?)</div>', str(review))
    rating_no_comment = re.search('Rated: <b>(.*?)</b>', str(review))
    rating_u= rating_no_comment.group(1) if rating_no_comment is not None else (rating_with_comment.group(1) if rating_with_comment is not None else NULL)
    reviews_df=reviews_df.append({'product_name': beer_company_name, \
                                'product_review': review_text.group(1) if review_text is not None else NULL,\
                                'user_rating': rating_u
                                }, ignore_index = True)

reviews_df
reviews_df[:5]


In [ ]:
#getting local reviews_df as the comments and analysis based on comment is changing
import pandas as pd
task_a_df = pd.read_csv ('dataforbeers.csv')
reviews_df=task_a_df

### **Task B**
Assume that a customer, who will be using this recommender system, has specified 3 attributes in a product. E.g., one website describes multiple attributes of beer:
https://www.dummies.com/food-drink/drinks/beer/beer-for-dummies-cheat-sheet/

Aggressive (Boldly assertive aroma and/or taste) 
Balanced: Malt and hops in similar proportions; equal representation of malt sweetness and hop bitterness in the flavor — especially at the finish
Complex: Multidimensional; many flavors and sensations on the palate
Crisp: Highly carbonated; effervescent
Fruity: Flavors reminiscent of various fruits or Hoppy: Herbal, earthy, spicy, or citric aromas and flavors of hops or Malty: Grainy, caramel-like; can be sweet or dry
Robust: Rich and full-bodied

Use the above attributes as examples only, for a word frequency analysis of beer reviews is a better way to find important attributes in the actual data.  

In [ ]:
import nltk
from nltk.corpus import stopwords 
nltk.download('stopwords')
nltk.download('punkt')

#defining a cleanup function to remove stopwords and posts whose length in less than 2 words
def cleanup(s):
 stopset = set(stopwords.words('english'))
 tokens = nltk.word_tokenize(s)
 cleanup = [token.lower() for token in tokens if token.lower() not in stopset and  len(token)>2]
 return cleanup

#remove rows where there is no review
reviews_df=reviews_df[reviews_df['product_review']!=0]

#convert to review to list of words
#not deduping for word frequencies
reviews_df['spilt_product_review']=reviews_df['product_review'].str.replace(".","").str.replace(",","").str.lower().apply(cleanup)

#get the word frequencies
word_frequencies=pd.DataFrame(reviews_df['spilt_product_review'].explode('spilt_product_review').value_counts())
word_frequencies[:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


,spilt_product_review
beer,1696
taste,1378
head,1231
good,960
like,806
one,777
dark,726
chocolate,711
sweet,689
nice,663


#### **Task C**

Perform a similarity analysis using cosine similarity (without word embeddings) with the 3 attributes specified by the customer and the reviews. 
The similarity script should accept as input a file with the product attributes, and calculate similarity scores (between 0 and 1) between these attributes and each review. That is, the output file should have 3 columns – product_name (for each product, the product_name will repeat as many times as there are reviews of the product), product_review and similarity_score. 

In [ ]:
#read user preference to a string

from google.colab import files
uploaded = files.upload()
#open text file in read mode
text_file = open("data.txt", "r")
user_pref = text_file.read().lower()
#close file

text_file.close()

print(user_pref)

Saving data.txt to data.txt
chocolate dark head



In [ ]:
#performing cosine similarity
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_cosine_similarity(review_string):
  review_string=review_string.lower()
  documents=[review_string,user_pref]
  count_vectorizer=CountVectorizer(stop_words='english')
  sparse_matrix=count_vectorizer.fit_transform(documents)
  doc_term_matrix=sparse_matrix.todense()
  df=pd.DataFrame(doc_term_matrix,columns=count_vectorizer.get_feature_names(),index=['x','y'])
  return cosine_similarity(df,df)[0,1] 

#keeping only required columns
reviews_df =reviews_df[['product_name','product_review']]
reviews_df['similarity_score']=reviews_df['product_review'].apply(get_cosine_similarity)
reviews_df.sort_values(by='similarity_score', ascending=False)[:5]

,product_name,product_review,similarity_score
11823,Speedway StoutAleSmith Brewing Company,Pours dark chocolate with khaki head and deep ...,0.662266
14355,Choklat (Blackwater Series)Southern Tier Brewi...,"Dark black, lots of dark chocolate taste and s...",0.560112
16563,Chocolate StoutRogue Ales,This beer is really good. Beautiful opaque loo...,0.547723
17883,Anchor PorterAnchor Brewing Company,Pours a very dark brownish color with a dark t...,0.545545
1945,MamanPerennial Artisan Ales,Bottle. Dark brown/black pour with a chocolate...,0.535303


#### **Task D**
For every review, perform a sentiment analysis (using VADER). In case you have to change the default values of words in the VADER lexicon, use this article: https://medium.com/swlh/adding-context-to-unsupervised-sentiment-analysis-7b6693d2c9f8 

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
!pip install vaderSentiment

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
reviews_df['scores'] = reviews_df['product_review'].apply(lambda review: sid.polarity_scores(review))
reviews_df['score_compound']  = reviews_df['scores'].apply(lambda score_dict: score_dict['compound'])
reviews_df=reviews_df.sort_values(by='score_compound', ascending=False)
reviews_df[:5]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,product_name,product_review,similarity_score,scores,score_compound
19915,Double Vanilla NoirPrairie Artisan Ales,"been wanting to try this one for awhile, loved...",0.132453,"{'neg': 0.035, 'neu': 0.708, 'pos': 0.257, 'co...",0.9987
2052,Nectarine Premierede Garde Brewing,this one has been on my most wanted list for a...,0.038749,"{'neg': 0.012, 'neu': 0.729, 'pos': 0.259, 'co...",0.9985
23139,Deadhead IPA Series: TourBusDESTIHL Brewery,"this is fabulous, perhaps the best hoppy beer ...",0.000000,"{'neg': 0.066, 'neu': 0.688, 'pos': 0.246, 'co...",0.9983
21509,Sugarwood BaklavaBoulevard Brewing Co.,well i saw this and thought it looked deliciou...,0.072452,"{'neg': 0.053, 'neu': 0.662, 'pos': 0.285, 'co...",0.9982
4148,Black TuesdayThe Bruery,Definitely one of those beers i've waited a lo...,0.061459,"{'neg': 0.013, 'neu': 0.789, 'pos': 0.198, 'co...",0.9978


#### **Task E**
Create an evaluation score for each beer that uses both similarity and sentiment scores. E.g., total score  = average of (similarity score + sentiment score) or a multiplicative model.

In [ ]:
#total score = average of (similarity score + sentiment score) 
#taking compound score as sentiment score NEED TO CONFIRM 

reviews_df['total_score'] = (reviews_df['similarity_score']+reviews_df['score_compound'])/2
reviews_df=reviews_df.sort_values(by='total_score', ascending=False)
reviews_df.sort_values(by='total_score', ascending=False)[:5]

,product_name,product_review,similarity_score,scores,score_compound,total_score
5012,ParabajavaFirestone Walker Brewing Co.,Extraordinarily rich and flavorful stout. Smel...,0.510754,"{'neg': 0.0, 'neu': 0.678, 'pos': 0.322, 'comp...",0.9517,0.731227
17892,Anchor PorterAnchor Brewing Company,A true American classic. This porter is quite ...,0.489898,"{'neg': 0.0, 'neu': 0.711, 'pos': 0.289, 'comp...",0.9589,0.724399
4880,Double ShotTree House Brewing Company,Poured from a can at fridge temp. Pours jet bl...,0.450988,"{'neg': 0.0, 'neu': 0.676, 'pos': 0.324, 'comp...",0.9869,0.718944
13316,Young's Double Chocolate StoutEagle Brewery,Looks great. Dark with smooth creamy head. The...,0.510754,"{'neg': 0.075, 'neu': 0.684, 'pos': 0.241, 'co...",0.9199,0.715327
14355,Choklat (Blackwater Series)Southern Tier Brewi...,"Dark black, lots of dark chocolate taste and s...",0.560112,"{'neg': 0.0, 'neu': 0.605, 'pos': 0.395, 'comp...",0.8689,0.714506


#### **Task F**

How would your recommendation change if you use word vectors (e.g., the spaCy package with medium sized pretrained word vectors) instead of the plain vanilla bag-of-words cosine similarity? One way to analyze the difference would be to consider the % of reviews that mention a preferred attribute. E.g., if you recommend a product, what % of its reviews mention an attribute specified by the customer? Do you see any difference across bag-of-words and word vector approaches? This article may be useful: https://medium.com/swlh/word-embeddings-versus-bag-of-words-the-curious-case-of-recommender-systems-6ac1604d4424?source=friends_link&sk=d746da9f094d1222a35519387afc6338
Note that the article doesn’t claim that bag-of-words will always be better than word embeddings for recommender systems. It lays out conditions under which it is likely to be the case. That is, depending on the attributes you use, you may or may not see the same effect.

In [ ]:
!python -m spacy download en_core_web_md

2022-10-01 15:58:17.319022: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 1.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
#spaCy uses word vectors for medium (md) and large (lg)
#!pip install Spacy
import spacy
nlp = spacy.load('en_core_web_md')

def get_spacy(review_string):
  review_string=review_string.lower()
  doc1 = nlp(review_string)
  doc2 = nlp(user_pref)
  #Calculates spaCy similarity between texts 1 and 2
  return doc1.similarity(doc2)

reviews_df['spacy_score']=reviews_df['product_review'].apply(get_spacy)
reviews_df.sort_values(by='spacy_score', ascending=False)[:5]

,product_name,product_review,similarity_score,scores,score_compound,total_score,spacy_score
17262,Big Bad BaptistEpic Brewing Company,"Dark dark thick back strap molasses color,whis...",0.518476,"{'neg': 0.0, 'neu': 0.843, 'pos': 0.157, 'comp...",0.6597,0.589088,0.724109
17516,Raspberry TartNew Glarus Brewing Company,"Poured very dark red color, smallish off-red h...",0.280056,"{'neg': 0.0, 'neu': 0.874, 'pos': 0.126, 'comp...",0.3182,0.299128,0.701792
16594,Chocolate StoutRogue Ales,Bomber from Albertsons. Near black pour with s...,0.481543,"{'neg': 0.095, 'neu': 0.782, 'pos': 0.122, 'co...",0.2023,0.341922,0.699376
7107,Heavy MettleTrillium Brewing Company,Smells dank and melon like. Straw colored haze...,0.120386,"{'neg': 0.071, 'neu': 0.785, 'pos': 0.144, 'co...",0.4215,0.270943,0.666374
15684,Cappuccino StoutLagunitas Brewing Company,"Dark ruby-black color, roasted caramel and cof...",0.195180,"{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'comp...",0.8442,0.519690,0.658144


# Task **G**

How would your recommendations differ if you ignored the similarity and feature sentiment scores and simply chose the 3 highest rated products from your entire dataset? Would these products meet the requirements of the user looking for recommendations? Why or why not? Justify your answer with analysis. Use the similarity and sentiment scores as well as overall ratings to answer this question.

In [ ]:
#Adding support to find the most popular beer ;  Should be rated at least 10 times with a comment.
dfforpopularbeers = task_a_df[task_a_df['product_name'].map(task_a_df['product_name'].value_counts()) > 10]
dfforpopularbeers['user_rating_f'] = dfforpopularbeers['user_rating'].astype(float)
grouped_multiple = dfforpopularbeers.groupby(['product_name'], as_index = False).agg({'user_rating_f': ['mean']})
grouped_multiple.columns = grouped_multiple.columns.droplevel()
grouped_multiple.sort_values(by=['mean'], ascending=False,inplace=True)
grouped_multiple[:3]

,,mean
379,Kentucky Brunch Brand StoutToppling Goliath Br...,4.82000
675,Vanilla Bean AssassinToppling Goliath Brewing ...,4.74500
66,Barrel Aged Imperial German Chocolate Cupcake ...,4.73325


The top three recommendations according to highly rated beers from the dataset are: 

1. Kentucky Brunch Brand StoutToppling Goliath Brewing Company
2. Vanilla Bean AssassinToppling Goliath Brewing Company
3. Barrel Aged Imperial German Chocolate Cupcake StoutAngry Chair Brewing


In [ ]:
#For the three beers, calculatin Similarity, Sentiment and Spacy scores for out given input by user
beer1_df = reviews_df[reviews_df['product_name'] == 'Kentucky Brunch Brand StoutToppling Goliath Brewing Company'].sort_values(by='spacy_score',ascending=False)
beer2_df = reviews_df[reviews_df['product_name'] == 'Vanilla Bean AssassinToppling Goliath Brewing Company'].sort_values(by='spacy_score',ascending=False)
beer3_df = reviews_df[reviews_df['product_name'] == 'Barrel Aged Imperial German Chocolate Cupcake StoutAngry Chair Brewing'].sort_values(by='spacy_score',ascending=False)

data = [beer1_df["total_score"], beer1_df["spacy_score"], beer2_df["total_score"], beer2_df["spacy_score"],beer3_df["total_score"], beer3_df["spacy_score"]]
headers = ["Beer 1 Total", "Beer 1 Spacy","Beer 2 Total", "Beer 2 Spacy","Beer 3 Total", "Beer 3 Spacy"]
df3 = pd. concat(data, axis=1, keys=headers)

df3.max(axis=0)


Beer 1 Total    0.580400
Beer 1 Spacy    0.535296
Beer 2 Total    0.603838
Beer 2 Spacy    0.578654
Beer 3 Total    0.615936
Beer 3 Spacy    0.589345
dtype: float64

As we can clearly see above, the recommendation which we do not make based on sentiment and similarity scores are not even matching 70% of the user requirement. The highest similarity score for our user specific requirements is Barrel Aged Imperial German Chocolate Cupcake StoutAngry Chair Brewing (beer 3, 61% by Similarity and Sentiment score and 58% by SpaCy), 11% lower than our previous recommendation of 72% by Spacy score for Big Bad BaptistEpic Brewing Company.
This was expected behaviour as the beer which would be popular overall would not be a perfect match for a particular user as preferences and criteria to like a beer vary vastly from a person to person.

# **Task H**

Using the top four attributes of beer (from word frequency analysis), calculate the lifts between these attributes and any 10 beers in your data. Choose one beer, and find the most similar beer (among the remaining 9) using the lift values. Explain your method

In [ ]:
#selecting top attributes from the dataset
top_attr= ['chocolate', 'sweet', 'dark', 'thick']
top_attr

['chocolate', 'sweet', 'dark', 'thick']

In [ ]:
# Selecting top 10 beers with highest comments for this analysis
import numpy as np
task_h_df = task_a_df[['product_name','product_review']].copy()
task_h_df['product_review'] = task_h_df['product_review'].replace(to_replace={'0':np.nan})
beer_10 = task_h_df.groupby('product_name')['product_review'].count().sort_values(ascending=False)[:10].index.to_list()
beer_10

['Perpetual IPATröegs Brewing Company',
 'Calf & HalfGreat Notion Brewing',
 'Banana Colada Melted Sno ConeSouthern Grist Brewing Co.',
 'King SueToppling Goliath Brewing Company',
 'Pabst Blue Ribbon (PBR)Pabst Brewing Company',
 'Centennial IPAFounders Brewing Company',
 'Pale AleSierra Nevada Brewing Co.',
 'Blind PigRussian River Brewing Company',
 'Heavy, Heavy, HeavyBeer Thug Brewing Company (BeerThugLife)',
 'Organic Chocolate StoutSamuel Smith Old Brewery (Tadcaster)']

In [ ]:
# Appending beer names (with spaces replaced by underscores) to review before calculating lifts of beer with attributes in reviews
task_h_df['product_review_beer'] = task_h_df['product_name'].str.replace(' ', '_') + ' ' + task_h_df['product_review'].astype(str)
task_h_df[task_h_df['product_name'].isin(beer_10)]['product_name']
task_h_df

,product_name,product_review,product_review_beer
0,Kentucky Brunch Brand StoutToppling Goliath Br...,NaN,Kentucky_Brunch_Brand_StoutToppling_Goliath_Br...
1,Kentucky Brunch Brand StoutToppling Goliath Br...,NaN,Kentucky_Brunch_Brand_StoutToppling_Goliath_Br...
2,Kentucky Brunch Brand StoutToppling Goliath Br...,NaN,Kentucky_Brunch_Brand_StoutToppling_Goliath_Br...
3,Kentucky Brunch Brand StoutToppling Goliath Br...,NaN,Kentucky_Brunch_Brand_StoutToppling_Goliath_Br...
4,Kentucky Brunch Brand StoutToppling Goliath Br...,I didnt think i was going to give it a perfect...,Kentucky_Brunch_Brand_StoutToppling_Goliath_Br...
...,...,...,...
23854,OasisTree House Brewing Company,NaN,OasisTree_House_Brewing_Company nan
23855,OasisTree House Brewing Company,NaN,OasisTree_House_Brewing_Company nan
23856,OasisTree House Brewing Company,Picked-up two cans at Tree House in Sandwich f...,OasisTree_House_Brewing_Company Picked-up two ...
23857,OasisTree House Brewing Company,NaN,OasisTree_House_Brewing_Company nan


In [ ]:
#calculating lift values for beers and top attributes
lift_beer=[]
lift_beer = pd.DataFrame(lift_beer)
beer_10 = [x.replace(' ', '_') for x in beer_10]

lift_array=[]
for brand1 in beer_10:
  for brand2 in top_attr:
    bb=0
    b1=0
    b2=0
    if brand1!=brand2:
      for token in task_h_df['product_review_beer']:
          if brand1 in token:
            b1=b1+1
          if brand2 in token:
            b2=b2+1
          if brand1 in token and brand2 in token:
            bb=bb+1
      lift_array.append([brand1,brand2,bb,b1,b2])      
      
lift_df = pd.DataFrame(columns=['beer', 'attribute','bb','b1','b2'], data=lift_array)  

#adding columns which will help in lift calculations
N=len(task_h_df)
lift_df['p_bb']=lift_df['bb']/N
lift_df['p_beer']=lift_df['b1']/N
lift_df['p_attribute']=lift_df['b2']/N
lift_df['lift']=lift_df['p_bb'] / (lift_df['p_attribute'] * lift_df['p_beer'])

k2k=pd.DataFrame(pd.crosstab(lift_df.beer, lift_df.attribute, values=lift_df.lift, aggfunc='mean'))
k3=k2k.copy()
k3

attribute,chocolate,dark,sweet,thick
beer,,,,
Banana_Colada_Melted_Sno_ConeSouthern_Grist_Brewing_Co.,0.000000,0.000000,1.553320,15.073375
Blind_PigRussian_River_Brewing_Company,0.000000,0.000000,0.000000,4.306679
Calf_&_HalfGreat_Notion_Brewing,5.130968,2.500943,2.329980,4.306679
Centennial_IPAFounders_Brewing_Company,0.000000,0.000000,1.553320,0.000000
"Heavy,_Heavy,_HeavyBeer_Thug_Brewing_Company_(BeerThugLife)",7.696452,6.252358,4.659961,6.460018
King_SueToppling_Goliath_Brewing_Company,0.000000,0.000000,1.553320,0.000000
Organic_Chocolate_StoutSamuel_Smith_Old_Brewery_(Tadcaster),14.110161,2.500943,3.106641,2.153339
Pabst_Blue_Ribbon_(PBR)Pabst_Brewing_Company,0.000000,0.000000,0.000000,0.000000
Pale_AleSierra_Nevada_Brewing_Co.,0.000000,1.250472,0.000000,0.000000


In [ ]:
#chosen beer: Calf_&_HalfGreat_Notion_Brewing
k3['Similarity to Calf_&_HalfGreat_Notion_Brewing'] = ((k3.loc['Calf_&_HalfGreat_Notion_Brewing']['chocolate']-k3['chocolate'])**2 )+ ((k3.loc['Calf_&_HalfGreat_Notion_Brewing']['dark']-k3['dark'])**2) + ((k3.loc['Calf_&_HalfGreat_Notion_Brewing']['sweet']-k3['sweet'])**2) + ((k3.loc['Calf_&_HalfGreat_Notion_Brewing']['thick']-k3['thick'])**2)
k3['Similarity to Calf_&_HalfGreat_Notion_Brewing'] = 1- (k3['Similarity to Calf_&_HalfGreat_Notion_Brewing'] /k3['Similarity to Calf_&_HalfGreat_Notion_Brewing'].abs().max())
k3

attribute,chocolate,dark,sweet,thick,Similarity to Calf_&_HalfGreat_Notion_Brewing
beer,,,,,
Banana_Colada_Melted_Sno_ConeSouthern_Grist_Brewing_Co.,0.000000,0.000000,1.553320,15.073375,0.000000
Blind_PigRussian_River_Brewing_Company,0.000000,0.000000,0.000000,4.306679,0.745079
Calf_&_HalfGreat_Notion_Brewing,5.130968,2.500943,2.329980,4.306679,1.000000
Centennial_IPAFounders_Brewing_Company,0.000000,0.000000,1.553320,0.000000,0.653052
"Heavy,_Heavy,_HeavyBeer_Thug_Brewing_Company_(BeerThugLife)",7.696452,6.252358,4.659961,6.460018,0.793969
King_SueToppling_Goliath_Brewing_Company,0.000000,0.000000,1.553320,0.000000,0.653052
Organic_Chocolate_StoutSamuel_Smith_Old_Brewery_(Tadcaster),14.110161,2.500943,3.106641,2.153339,0.424130
Pabst_Blue_Ribbon_(PBR)Pabst_Brewing_Company,0.000000,0.000000,0.000000,0.000000,0.620688
Pale_AleSierra_Nevada_Brewing_Co.,0.000000,1.250472,0.000000,0.000000,0.652149


To calculate the similarity between one of these beers and the remaining 9, we have selected Calf & Half as the beer and calculated the similarity score of the other 9 beers with respect to these four attributes.
To calculate the similarity, we've taken a summation of the squares of deviation from the lift value for our beer, normalized this value and subtracted from 1 to determine the measure of similarity to our beer. Based on our analysis, Calf & Half and Heavy, Heavy, Heavy	are the most similar among these 10 beers as their lift values differ by least for these 4 attributes and as seen above the similarity score to Calf & Half is the highest as well. On the contrary, Banana Colada Melted Sno Cone is the most contrasting beer to the Calf & Half as it has the lowest similarity score.